In [ ]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.cleanup import *
import time
time1 = time.time()

my_auth = get_key('koray_data')

all_users = ff_utils.search_metadata('/search/?type=user', my_auth)
print(len(all_users))

f = open('/Users/koray/downloads/hms-dbmi.json')
dicto = json.loads(f.read())
print(len(dicto))


In [ ]:
dicto = [i for i in dicto if i.get('Email')]
print(len(dicto))
all_data = []
all_cs = []
found = 0
nfound = 0
for i in all_users:
    em = i['email']
    
    match = False
    cs = []
    count = 0
    for x in dicto:
        em2 = x['Email']
        c = x['Connection']
        if em.lower().strip() == em2.lower().strip():
            #print(em2, c)
            match = True
            cs.append(c)
            log = x['Logins Count']
            count += log
    cs = sorted(cs)
    
    cs = ' | '.join(cs)
    all_cs.append(cs)
    

    if match:
        found += 1
        all_data.append({'mail':em, 'count':count, 'services': cs})
    else:
        nfound += 1
print(found, nfound)


In [ ]:
from collections import Counter
print(Counter(all_cs))

In [ ]:
all_data = sorted(all_data, key=lambda k:[k['count']], reverse=True)

In [ ]:
for i in all_data:
    print("{:<35} {}".format(i['mail'], i['count']))

In [ ]:
def write_outfile(terms, filename, pretty=False):
    '''terms is a list of dicts
        write to file by default as a json list or if pretty
        then same with indents and newlines
    '''
    with open(filename, 'w') as outfile:
        if pretty:
            json.dump(terms, outfile, indent=4)
        else:
            json.dump(terms, outfile)
            
            
            
            
            
listem = [{"a": "araba", "b": "badana"}, {"a": "adana", "b": "balaba"}]

write_outfile(listem, 'bir.json', pretty=False)
write_outfile(listem, 'birim.json', pretty=True)


In [ ]:
opf_set = ('search/?type=ExperimentSet&other_processed_files.title%21=No+value&field=status'
               '&field=other_processed_files&field=experiments_in_set.other_processed_files')
opf_exp = ('search/?type=ExperimentSet&other_processed_files.title=No+value'
           '&experiments_in_set.other_processed_files.title%21=No+value'
           '&field=experiments_in_set.other_processed_files&field=status')
    
res_set = ff_utils.search_metadata(opf_set, my_auth)
res_exp = ff_utils.search_metadata(opf_exp, my_auth)
print(len(res_set))
print(len(res_exp))

In [ ]:
import time


from dcicutils import ff_utils
from dcicutils import es_utils
files = []

all_sets = res_set + res_exp

for a_set in all_sets:
    if a_set.get('other_processed_files'):
        for case in a_set['other_processed_files']:
            files.extend([i['uuid'] for i in case['files']])
    if a_set.get('experiments_in_set'):
        for an_exp in a_set['experiments_in_set']:
            for case in an_exp['other_processed_files']:
                files.extend([i['uuid'] for i in case['files']])
print(len(files))

for chunk in [100, 500, 1000, 2000, 5000]:
    time1 = time.time()
    resp =  ff_utils.get_es_metadata(files, chunk_size=chunk, key=my_auth)
    time2 = time.time()
    print(round((time2-time1), 1), 'sec for collection', chunk)

In [ ]:
import time
time1 = time.time()

files = 'search/?type=FileProcessed&field=status'
file_results = ff_utils.search_metadata(files + '&experiment_sets.uuid!=No+value', ff_env='data')
file_results += ff_utils.search_metadata(files + '&experiments.uuid!=No+value', ff_env='data')
file_results += ff_utils.search_metadata(files + '&status=deleted&status=replaced&experiment_sets.uuid!=No+value',
                                         ff_env='data')
file_results += ff_utils.search_metadata(files + '&status=deleted&status=replaced&experiments.uuid!=No+value',
                                         ff_env='data')
print(len(file_results))

time2 = time.time()
print(round((time2-time1), 1), 'sec for collection')